# Install package

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)




/bin/bash: nvidia-smi: command not found


In [ ]:
!pip install mat4py
from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive


In [ ]:
# A RNN-based Reinforcement Learning Framework for Frequency Control Problem with Stability Guarantee
import collections
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import gym
import tensorflow as tf
import tensorflow_probability as tfp

import os
import random
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.layers import RNN
import tensorflow.keras.backend as K
import sys
from gym import spaces
from gym.utils import seeding
import copy
from mat4py import loadmat
from tensorflow.keras import layers
from tensorflow import keras
import networkx as nx
import scipy
import pickle
import time


/usr/local/lib/python3.10/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <


In [ ]:
import random
from datetime import datetime
random.seed(datetime.now())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-4-ffa94a10b395>:3: DeprecationWarning: Seeding based on hashing is deprecated
since Python 3.9 and will be removed in a subsequent version. The only 
supported seed types are: None, int, float, str, bytes, and bytearray.
  random.seed(datetime.now())


# Environment Setup

In [ ]:
# Frequency Control Porblem Environment
class Frequency(gym.Env):
    def  __init__(self, Pm,M,D,F,delta_t,max_action,dim_action,Penalty_action, coef_cost):
        self.param_gamma = 1
        self.sigma_e=0.05
        self.M = M
        self.D = D
        self.Pm = Pm
        self.max_action = max_action
        self.dim_action = dim_action
        self.omega_scale = 2*np.pi# this change the unit of omega to Hz
        self.viewer  =  None
        self.state_x = []
        self.delta_t = delta_t
        self.Penalty_action = Penalty_action
        self.state_x_transfer1 = np.vstack((np.hstack((np.identity(dim_action,dtype = np.float32),np.zeros((dim_action,dim_action),dtype = np.float32))),\
                                np.hstack((delta_t*self.omega_scale*np.identity(dim_action,dtype = np.float32),\
                                           np.identity(dim_action,dtype = np.float32)-delta_t*np.diag(D/M)))))

        self.state_x_transferF = -delta_t*(((M**(-1)).reshape(dim_action,1))@np.ones((1,dim_action),dtype = np.float32))*F
        self.state_x_transfer2 = np.hstack((np.zeros((dim_action,dim_action),dtype = np.float32),\
                                        np.identity(dim_action,dtype = np.float32)))

        self.state_x_transfer3 = np.hstack((np.zeros((1,dim_action),dtype = np.float32),\
                                        delta_t*Pm*(M**(-1))))
        self.state_x_transfer3_Pm = np.hstack((np.zeros((dim_action,dim_action),dtype = np.float32),\
                                        delta_t*np.diag((M**(-1)))))
        self.state_x_transfer4 = np.hstack((np.zeros((dim_action,dim_action),dtype = np.float32),\
                                        -delta_t*np.diag((M**(-1)))))

        self.select_add_w = np.vstack((np.zeros((dim_action,1),dtype = np.float32),\
                                        np.ones((dim_action,1),dtype = np.float32)))
        self.select_w = np.vstack((np.zeros((dim_action,dim_action),dtype = np.float32),\
                                        np.identity(dim_action,dtype = np.float32)))
        self.select_delta = np.vstack((np.identity(dim_action,dtype = np.float32),\
                                        np.zeros((dim_action,dim_action),dtype = np.float32)))
        self.diag_c  =  np.diag(coef_cost)

        # np.identity(dim_action,dtype = np.float32)


    def step(self, action,Pm):

        cost_action, grad_action  =  self.calc_grad_action(action)
        dot_s_new  =  self.state_x@self.select_w-\
                np.sum(np.transpose(grad_action)@np.ones((1,dim_action),dtype = np.float32)\
                - np.ones((dim_action,1),dtype = np.float32)@grad_action,axis = 1)@self.diag_c*0.05
        self.state_x  =  copy.deepcopy(self.state_x@self.state_x_transfer1\
              + np.sum(np.sin( np.transpose(self.state_x@self.select_delta)@np.ones((1,dim_action),dtype = np.float32)-\
                np.ones((dim_action,1),dtype = np.float32)@(self.state_x@self.select_delta))*self.state_x_transferF,axis = 1 )\
                      @self.state_x_transfer2\
              + Pm@self.state_x_transfer3_Pm\
                          +action@self.state_x_transfer4)
        self.state_s  =  self.state_s+self.delta_t*dot_s_new/1
        loss  =  self.param_gamma*pow(self.state_x,2)@self.select_add_w
        return self.state_s, self.state_x, loss



    def step_PI_CommEdge_WoCost(self, actionP, actionI,Pm):

        action  =  actionP + actionI
        dot_s_new  =  self.state_x@self.select_w
        self.state_x  =  copy.deepcopy(self.state_x@self.state_x_transfer1\
              + np.sum(np.sin( np.transpose(self.state_x@self.select_delta)@np.ones((1,dim_action),dtype = np.float32)-\
                np.ones((dim_action,1),dtype = np.float32)@(self.state_x@self.select_delta))*self.state_x_transferF,axis = 1 )\
                      @self.state_x_transfer2\
              + Pm@self.state_x_transfer3_Pm\
                          +action@self.state_x_transfer4)
        self.state_s  =  self.state_s+self.delta_t*dot_s_new/1
        loss  =  self.param_gamma*pow(self.state_x,2)@self.select_add_w
        return self.state_s, self.state_x, loss


    def set_state(self, state_input):
        self.state_x = state_input
        self.state_s  =  np.zeros((1,dim_action),dtype = np.float32)


    def reset(self):
        initial_state1 = np.random.uniform(0.0,0.3,(1,self.dim_action))
        initial_state2 = np.random.uniform(-0.03,0.03,(1,self.dim_action))
        s_concate = np.hstack((initial_state1,initial_state2)).astype(np.float32)
        self.state_x  =  s_concate
        self.state_s  =  np.zeros((1,dim_action),dtype = np.float32)

        return self.state_x




In [ ]:
# Simulation data load from IEEE 39-bus system
data = loadmat('IEEE_39bus_Kron_new.mat')

K_EN=data['Kron_39bus']['K']
K_EN=np.asarray(K_EN, dtype=np.float32)

H=data['Kron_39bus']['H']
H=np.asarray(H, dtype=np.float32)

Damp=data['Kron_39bus']['D']
Damp=np.asarray(Damp, dtype=np.float32)

omega_R=data['Kron_39bus']['omega_R']

A_EN=data['Kron_39bus']['A']
A_EN=np.asarray(A_EN, dtype=np.float32)

gamma=data['Kron_39bus']['gamma']
gamma=np.asarray(gamma, dtype=np.float32)

In [ ]:
dim_action = 10 #dimension of action space
dim_state = 2*dim_action #dimension of state space
action_units=dim_action
delta_t=0.01
M=H.reshape(dim_action)*2/omega_R*2*np.pi
# D=Damp.reshape(dim_action)/omega_R*2*np.pi
D=np.zeros(dim_action,dtype=np.float32)
D[0]=2*590/100
D[1:8]=2*865/100
D[8:10]=2*911/100;
D=D/omega_R*2*np.pi
F=K_EN
Penalty_action=0.01*0.2
Pm=np.array([[-0.19983394, -0.25653884, -0.25191885, -0.10242008, -0.34510365,
         0.23206371,  0.4404325 ,  0.5896664 ,  0.26257738, -0.36892462]],dtype=np.float32)

max_action=np.array([[0.19606592, 0.2190382 , 0.22375287, 0.0975513 , 0.29071101,
        0.22091283, 0.38759459, 0.56512538, 0.24151538, 0.29821917]],dtype=np.float32)*5
equilibrium_init=np.array([[ -0.05420687, -0.07780334, -0.07351729, -0.05827823, -0.09359571,
        -0.02447385, -0.00783582,  0.00259523, -0.0162409 , -0.06477749,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.       ]],dtype=np.float32)
coef_cost = np.array([0.94162537, 1.15464676, 0.55487802, 0.71176656, 1.27476119,
       1.02529959, 1.2946042 , 1.01800112, 0.99465694, 0.73467357], dtype=np.float32)
# np.random.uniform(0.5,1.5,(dim_action))
env = Frequency(Pm,M,D,F,delta_t,max_action,dim_action,Penalty_action, coef_cost)


# Train

### Neural-PI

In [ ]:
# RNN Cell to integrate state transition dynamics
class MinimalRNNCell_WoCost_SCNN(keras.layers.Layer):

    def __init__(self, units, action_units_node_p, action_units_node_i, internal_units,internal_units_SCNN, env,batchsize,**kwargs):
        self.units = units
        self.state_size = units
        self.action_units_node_p = action_units
        self.action_units_node_i = action_units
        self.internal_units = internal_units
        self.internal_units_SCNN = internal_units_SCNN
        self.batchsize = batchsize
        self.delta_t = tf.constant(env.delta_t,dtype = tf.float32)

        self.state_x_transfer1 = tf.constant(env.state_x_transfer1,dtype = tf.float32)
        self.state_x_transferF = tf.constant(env.state_x_transferF,dtype = tf.float32)
        self.state_x_transfer2 = tf.constant(env.state_x_transfer2,dtype = tf.float32)
        self.state_x_transfer3 = tf.constant(env.state_x_transfer3,dtype = tf.float32)
        self.state_x_transfer4 = tf.constant(env.state_x_transfer4,dtype = tf.float32)
        self.state_x_transfer3_Pm = tf.constant(env.state_x_transfer3_Pm,dtype = tf.float32)
        self.select_add_w = tf.constant(env.select_add_w,dtype = tf.float32)
        self.select_w = tf.constant(env.select_w,dtype = tf.float32)
        self.select_delta = tf.constant(env.select_delta,dtype = tf.float32)
        self.max_action = tf.constant(env.max_action,dtype = tf.float32)
        self.w_recover = tf.constant(tf.linalg.band_part(-tf.ones((internal_units,internal_units)),0,1)\
                                        +2*tf.eye(internal_units),dtype = tf.float32)
        self.b_recover = tf.constant(tf.linalg.band_part(tf.ones((internal_units,internal_units)),0,-1)\
                                        -tf.eye(internal_units),dtype = tf.float32)
        self.diag_c = tf.constant(env.diag_c, dtype = tf.float32)
############ PI controller :P

        self.Multiply_ones_node_p = tf.tile(tf.ones((action_units_node_p,action_units_node_p),dtype=np.float32)[None], [batchsize, 1, 1])
        self.ones_node_p = tf.ones((action_units_node_p,internal_units),dtype=tf.float32)

############ PI controller :I
        self.Multiply_ones_node_i = tf.tile(tf.ones((action_units_node_i,action_units_node_i),dtype=np.float32)[None], [batchsize, 1, 1])
        self.ones_node_i = tf.ones((action_units_node_i,internal_units),dtype=tf.float32)
        self.obs_zeros = tf.zeros((1, action_units_node_p))


        super(MinimalRNNCell_WoCost_SCNN, self).__init__(**kwargs)

    def build(self, input_shape):
############# PI controller  : P
        self.W_p1 =  self.add_weight(
            shape=(self.action_units_node_p,self.internal_units_SCNN),
            initializer='random_normal',
            trainable=True,
            name='W_p1')
        self.b_p1 =  self.add_weight(
            shape=(self.internal_units_SCNN,),
            initializer='random_normal',
            trainable=True,
            name='b_p1')
        self.W_p2 =  self.add_weight(
            shape=(self.action_units_node_p,self.internal_units_SCNN),
            initializer='random_normal',
            trainable=True,
            name='W_p2')
        self.W_pz1 =  self.add_weight(
            shape=(self.internal_units_SCNN,self.internal_units_SCNN),
            initializer='random_normal',
            trainable=True,
            constraint = tf.keras.constraints.non_neg(),
            name='W_pz1')
        self.b_p2 =  self.add_weight(
            shape=(self.internal_units_SCNN,),
            initializer='random_normal',
            trainable=True,
            name='b_p2')

        self.W_p3 =  self.add_weight(
            shape=(self.action_units_node_p,1),
            initializer='random_normal',
            trainable=True,
            name='W_p3')
        self.W_pz2 =  self.add_weight(
            shape=(self.internal_units_SCNN,1),
            initializer='uniform',
            trainable=True,
            constraint = tf.keras.constraints.non_neg(),
            name='W_pz2')
        self.b_p3 =  self.add_weight(
            shape=(1,),
            initializer='random_normal',
            trainable=True,
            name='b_p3')

        ######################### integral
        self.W_i1 =  self.add_weight(
            shape=(self.action_units_node_p,self.internal_units_SCNN),
            initializer='random_normal',
            trainable=True,
            name='W_i1')
        self.b_i1 =  self.add_weight(
            shape=(self.internal_units_SCNN,),
            initializer='random_normal',
            trainable=True,
            name='b_i1')
        self.W_i2 =  self.add_weight(
            shape=(self.action_units_node_p,self.internal_units_SCNN),
            initializer='random_normal',
            trainable=True,
            name='W_i2')
        self.W_iz1 =  self.add_weight(
            shape=(self.internal_units_SCNN,self.internal_units_SCNN),
            initializer='uniform',
            trainable=True,
            constraint = tf.keras.constraints.non_neg(),
            name='W_iz1')
        self.b_i2 =  self.add_weight(
            shape=(self.internal_units_SCNN,),
            initializer='random_normal',
            trainable=True,
            name='b_i2')

        self.W_i3 =  self.add_weight(
            shape=(self.action_units_node_p,1),
            initializer='random_normal',
            trainable=True,
            name='W_i3')
        self.W_iz2 =  self.add_weight(
            shape=(self.internal_units_SCNN,1),
            initializer='uniform',
            trainable=True,
            constraint = tf.keras.constraints.non_neg(),
            name='W_iz2')
        self.b_i3 =  self.add_weight(
            shape=(1,),
            initializer='random_normal',
            trainable=True,
            name='b_i3')


        self.w_beta =  self.add_weight(
            # shape=(1),
            initializer=tf.constant_initializer(1),
            trainable=True,
            name='beta')

        self.built = True

    @tf.function
    def SCNN_action(self, obs):

        with tf.GradientTape() as tape:
            tape.watch(obs)
            z1 = self.softplus_beta(K.dot(obs, self.W_p1)+ self.b_p1)
            z2 = self.softplus_beta(K.dot(obs, self.W_p2)+K.dot(z1, self.W_pz1)+ self.b_p2)
            z3 = self.softplus_beta(K.dot(obs, self.W_p3)+K.dot(z2, self.W_pz2)+ self.b_p3)
        action_node_p = tf.squeeze(tape.batch_jacobian(z3, obs))
        return action_node_p

    @tf.function
    def SCNN_actionI(self, obs):

        with tf.GradientTape() as tape:
            tape.watch(obs)
            z1 = self.softplus_beta(K.dot(obs, self.W_i1)+ self.b_i1)
            z2 = self.softplus_beta(K.dot(obs, self.W_i2)+K.dot(z1, self.W_iz1)+ self.b_i2)
            z3 = self.softplus_beta(K.dot(obs, self.W_i3)+K.dot(z2, self.W_iz2)+ self.b_i3)
        action_node_i = 1*tf.squeeze(tape.batch_jacobian(z3, obs))

        return action_node_i

    @tf.function
    def softplus_beta(self, x):

        return K.softplus(self.w_beta*x)/(self.w_beta)

    @tf.function
    def call(self, inputs, states):
        # stacked ReLU structure to represent control network
        prev_state = states[0]
        prev_state_x = prev_state[:,0:self.action_units_node_p*2]
        prev_state_s =  prev_state[:,self.action_units_node_p*2:self.action_units_node_p*3]
##################### PI: P

        obs_y = K.dot(prev_state_x,self.select_w)
        action_node_p = self.SCNN_action(obs_y)-self.SCNN_action(self.obs_zeros)


##################### PI: I

        action_node_i = self.SCNN_actionI(prev_state_s)-self.SCNN_actionI(self.obs_zeros)


###########################
        action_nonconstrain = action_node_p + action_node_i
        action =  self.max_action-K.relu(self.max_action-action_nonconstrain)+K.relu(-self.max_action-action_nonconstrain)
        #
#########################
        # calculate state on s
        dot_s = K.dot(prev_state_x,self.select_w)

#######################
        # integrate the state transition dynamics
        new_state_x = K.dot(prev_state_x, self.state_x_transfer1)+\
            K.dot(K.sum(K.sin( K.dot(tf.linalg.diag(K.dot(prev_state_x, self.select_delta)),tf.ones((dim_action,dim_action),dtype = np.float32))-\
                                tf.matmul(self.Multiply_ones_node_p,tf.linalg.diag(K.dot(prev_state_x, self.select_delta))))\
                        *self.state_x_transferF,axis = 2 )\
                                      ,self.state_x_transfer2)\
                             + self.state_x_transfer3+K.dot(action,self.state_x_transfer4)\
                             + inputs@self.state_x_transfer3_Pm


        loss0 = K.dot(K.pow(new_state_x,2),self.select_add_w)
        frequency = K.dot(new_state_x,self.select_w)
        new_state_s = prev_state_s + self.delta_t*dot_s
        next_state = tf.concat([new_state_x,  new_state_s], axis = 1)
        return [loss0,frequency,action_node_p,action_node_i, action], [next_state]




In [ ]:
loop_seed = 5
model_list = []
loss_list = []
episodes  = 400 # total number of iterations to update weights
units = dim_action*3 #dimension of each state
internal_units = 20 # demension of the neural network for control policy
internal_units_SCNN = 20
T = 400  #Total period considered
Batch_num = 300
# Batch_num = 300 # number of batch in each episodes
PrintUpdate = 50
Comp_time_list = []

for loop in range(3,loop_seed):
    random.seed(datetime.now())

    start = time.time()

    cell_WoCost_SCNN = MinimalRNNCell_WoCost_SCNN(units,dim_action, dim_action, internal_units, internal_units_SCNN, env,Batch_num)
    layer_WoCost_SCNN = RNN(cell_WoCost_SCNN,return_sequences = True,stateful = True)
    input_1 = tf.keras.Input(batch_shape = (Batch_num,T,dim_action))
    outputs = layer_WoCost_SCNN((input_1))
    model_WoCost_SCNN = tf.keras.models.Model([input_1], outputs)
    model_WoCost_SCNN.compile(optimizer = 'adam', loss = 'mse', metrics = ['accuracy'])

    x0 = np.ones((Batch_num,T,dim_action))
    y0 = model_WoCost_SCNN(x0)
    Loss_record = []
    Pe_rnn_record = []
    global_step = tf.Variable(0, trainable = False)
    learning_rate_initial = 0.05

    decayed_lr  = tf.keras.optimizers.schedules.ExponentialDecay(
        learning_rate_initial, 50, 0.7, staircase = True)
    optimizer = tf.keras.optimizers.Adam(learning_rate = decayed_lr)

    delta_rnn_init_bound = 0.05*0
    omega_rnn_init_bound = 0.1*0 #in Hz
    num_gen_step = 3
    Percent_step_change = 1
    range_step_change = 1
    for i in range(0,episodes):
        start_in = time.time()
        initial_state = np.zeros((Batch_num,action_units*2))+equilibrium_init
        Pm_change = np.zeros((Batch_num,T,dim_action))
        for gen_interupt in range(0, num_gen_step):
            idx_gen_deviation = np.random.randint(0, action_units, Batch_num*Percent_step_change)
            idx_batch_deviation = np.random.randint(0, Batch_num, Batch_num*Percent_step_change)
            slot_start_deviation = np.random.randint(0, T/20, Batch_num*Percent_step_change)
            step_change = np.random.uniform(-1,1,(Batch_num*Percent_step_change))*range_step_change
            for t_interupt in range(0,T):
                Pm_change[idx_batch_deviation,t_interupt, idx_gen_deviation]\
                                = (slot_start_deviation<= t_interupt)*step_change
        layer_WoCost_SCNN.reset_states( np.hstack((initial_state, np.zeros((Batch_num,action_units)))))
        with tf.GradientTape(persistent = True) as tape:
            [loss0,frequency,action0,actions, action] = model_WoCost_SCNN(Pm_change)
            loss_action =0.005*K.sum(K.pow(action,2))/Batch_num
            loss_freq = K.sum(K.max(K.abs(frequency),axis = 1))/Batch_num + 0.05*K.sum(K.abs(frequency))/Batch_num
            loss = loss_action + loss_freq

        grads = tape.gradient(loss, model_WoCost_SCNN.variables)
        optimizer.apply_gradients(zip(grads, model_WoCost_SCNN.variables))
        Loss_record.append(loss)

        if i % (PrintUpdate) ==  0:
            print('episode',i, 'Loss',loss)
            print('episode',i, 'Loss_frequency',loss_freq)
            print('            time,',  time.time()- start_in  )

    end = time.time()
    print(end - start)
    Comp_time_list.append(end - start)

    model_list.append(model_WoCost_SCNN)
    loss_list.append(np.array(Loss_record))

print('computation time ', np.array(Comp_time_list))
print('computation time mean', np.mean(np.array(Comp_time_list)))

In [ ]:
plt.plot(Loss_record)
plt.xlabel('episoid')
plt.ylabel('Loss')


### Linear-PI

In [ ]:
# RNN Cell to integrate state transition dynamics
class MinimalRNNCell_WoCost_Linear(keras.layers.Layer):

    def __init__(self, units, action_units_node_p, action_units_node_i, internal_units,internal_units_ICNN, env,batchsize,**kwargs):
        self.units = units
        # self.action_units = action_units
        self.state_size = units
        self.action_units_node_p = action_units
        self.action_units_node_i = action_units
        self.internal_units = internal_units
        self.internal_units_ICNN = internal_units_ICNN
        self.batchsize = batchsize
        self.delta_t = tf.constant(env.delta_t,dtype = tf.float32)

        self.state_x_transfer1 = tf.constant(env.state_x_transfer1,dtype = tf.float32)
        self.state_x_transferF = tf.constant(env.state_x_transferF,dtype = tf.float32)
        self.state_x_transfer2 = tf.constant(env.state_x_transfer2,dtype = tf.float32)
        self.state_x_transfer3 = tf.constant(env.state_x_transfer3,dtype = tf.float32)
        self.state_x_transfer4 = tf.constant(env.state_x_transfer4,dtype = tf.float32)
        self.state_x_transfer3_Pm = tf.constant(env.state_x_transfer3_Pm,dtype = tf.float32)
        self.select_add_w = tf.constant(env.select_add_w,dtype = tf.float32)
        self.select_w = tf.constant(env.select_w,dtype = tf.float32)
        self.select_delta = tf.constant(env.select_delta,dtype = tf.float32)
        self.max_action = tf.constant(env.max_action,dtype = tf.float32)
        # self.Multiply_ones = tf.tile(tf.ones((action_units,action_units),dtype = np.float32)[None], [batchsize, 1, 1])
        self.w_recover = tf.constant(tf.linalg.band_part(-tf.ones((internal_units,internal_units)),0,1)\
                                        +2*tf.eye(internal_units),dtype = tf.float32)
        self.b_recover = tf.constant(tf.linalg.band_part(tf.ones((internal_units,internal_units)),0,-1)\
                                        -tf.eye(internal_units),dtype = tf.float32)
        # self.ones_frequency = tf.ones((action_units,internal_units),dtype = tf.float32)
        self.diag_c = tf.constant(env.diag_c, dtype = tf.float32)
        self.incidence_communication = tf.constant(env.incidence_communication,dtype = tf.float32)
   ############ PI controller :P

        # self.max_action_node_p = tf.constant(env.max_action_node_p,dtype=tf.float32)
        self.Multiply_ones_node_p = tf.tile(tf.ones((action_units_node_p,action_units_node_p),dtype=np.float32)[None], [batchsize, 1, 1])
        self.ones_node_p = tf.ones((action_units_node_p,internal_units),dtype=tf.float32)

        self.collect_W_p = []
        self.collect_W_i = []

        super(MinimalRNNCell_WoCost_Linear, self).__init__(**kwargs)

    def build(self, input_shape):
############# PI controller  : P
        for comm_subset in Comm_set:
            subset_dim = np.shape(comm_subset)[0]
            self.W_p =  self.add_weight(
                shape=(self.subset_dim,self.subset_dim),
                initializer='random_normal',
                trainable=True,
                name='W_p')
            self.collect_W_p.append(self.W_p)


            ######################### integral
            self.W_i =  self.add_weight(
                shape=(self.subset_dim,self.subset_dim),
                initializer='random_normal',
                trainable=True,
                name='W_i')
            self.collect_W_i.append(self.W_i)




        self.built = True

    @tf.function
    def action_linear_p_subset(self, obs):
        action =
        for comm_subset in Comm_set:
            subset_idx = comm_subset
            action += K.dot(obs_y[:,subset_idx], self.W_p)


    @tf.function
    def call(self, inputs, states):
        # stacked ReLU structure to represent control network
        prev_state = states[0]
        prev_state_x = prev_state[:,0:self.action_units_node_p*2]
        prev_state_s =  prev_state[:,self.action_units_node_p*2:self.action_units_node_p*3]


##################### PI: P

        obs_y = K.dot(prev_state_x,self.select_w)

        action_node_p = K.dot(obs_y, self.W_p)


##################### PI: I

        action_node_i = K.dot(prev_state_s,  self.W_i)


###########################
        action_nonconstrain = action_node_p + action_node_i
        action =  self.max_action-K.relu(self.max_action-action_nonconstrain)+K.relu(-self.max_action-action_nonconstrain)
        #
#########################
        # calculate state on s
        dot_s = K.dot(prev_state_x,self.select_w)

#######################
        # integrate the state transition dynamics
        new_state_x = K.dot(prev_state_x, self.state_x_transfer1)+\
            K.dot(K.sum(K.sin( K.dot(tf.linalg.diag(K.dot(prev_state_x, self.select_delta)),tf.ones((dim_action,dim_action),dtype = np.float32))-\
                                tf.matmul(self.Multiply_ones_node_p,tf.linalg.diag(K.dot(prev_state_x, self.select_delta))))\
                        *self.state_x_transferF,axis = 2 )\
                                      ,self.state_x_transfer2)\
                             + self.state_x_transfer3+K.dot(action,self.state_x_transfer4)\
                             + inputs@self.state_x_transfer3_Pm


        loss0 = K.dot(K.pow(new_state_x,2),self.select_add_w)
        frequency = K.dot(new_state_x,self.select_w)
        new_state_s = prev_state_s + self.delta_t*dot_s
        next_state = tf.concat([new_state_x,  new_state_s], axis = 1)
        return [loss0,frequency,action_node_p,action_node_i, action], [next_state]




In [ ]:
loop_seed = 5
PI_Linear_list = []
loss_list = []
Comp_time_list = []


PrintUpdate = 100
episodes =400 # total number of iterations to update weights


units = dim_action*2 + dim_edge #dimension of each state
internal_units=20 # demension of the neural network for control policy
internal_units_Linear = 20

T = 300  #Total period considered
Batch_num=300 # number of batch in each episodes
PrintUpdate=1

for loop in range(0,loop_seed):
    random.seed(datetime.now())
    start = time.time()


    cell = MinimalRNNCell_Linear(units, dim_edge ,dim_action, dim_action, internal_units, internal_units_Linear,env,Batch_num)
    layer = RNN(cell,return_sequences=True,stateful = True)
    input_1 = tf.keras.Input(batch_shape=(Batch_num,T,units))
    outputs = layer((input_1))
    model_Linear = tf.keras.models.Model([input_1], outputs)
    model_Linear.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

    x0=np.ones((Batch_num,T,units))
    y0=model_Linear(x0)
    Loss_record_Linear=[]
    Pe_rnn_record=[]
    global_step = tf.Variable(0, trainable=False)
    learning_rate_initial=0.05
    decayed_lr =tf.keras.optimizers.schedules.ExponentialDecay(
        learning_rate_initial, 50, 0.7, staircase=True)
    optimizer=tf.keras.optimizers.Adam(learning_rate=decayed_lr)


    for i in range(0,episodes):
        start_in = time.time()

        initial_state1 = v0_nom*np.ones((Batch_num,dim_action)) + v0_random*np.random.uniform(0,1.,(Batch_num, dim_action))
        initial_state2 = env.eta0*np.ones((Batch_num,dim_edge))
        initial_state3 = np.zeros((Batch_num, dim_action))
        initial_state = np.hstack((initial_state1, initial_state2, initial_state3))
        layer.reset_states( initial_state)
        with tf.GradientTape(persistent=True) as tape:
            [new_state_v, new_state_eta, action_edge, action_P, action_I]=model_Linear(x0)


    ##################
            loss_state1 =1* K.sum(K.relu(-new_state_eta+1))/Batch_num/units

            state_v_minus_ref = new_state_v-env.state_v_ref
            loss_state3 = K.sum(K.abs(state_v_minus_ref))/Batch_num/units
            loss_action_edge = 0.01*K.sum(K.pow(action_edge, 2))/Batch_num/units
            loss_action_w = 0.05*K.sum(K.pow(action_P+action_I,2)@env.diag_c)/Batch_num/units
            loss = (loss_state3 + loss_action_edge + loss_action_w)


        grads = tape.gradient(loss, model_Linear.variables)
        optimizer.apply_gradients(zip(grads, model_Linear.variables))
        Loss_record_Linear.append(loss)
        if i % (PrintUpdate) == 0:
            print('episode',i, 'Loss',loss)
            print('   Loss_v_minus_ref',loss_state3, 'Loss_eta',loss_state1  )
            print('   Loss_u_edge', loss_action_edge, 'loss_action_w', loss_action_w )
            print('   up/ui',K.sum(K.pow(action_P,2)@env.diag_c)/K.sum(K.pow(action_I,2)@env.diag_c))
            print('            time,',  time.time()- start_in  )



    end = time.time()
    print(end - start)
    Comp_time_list.append(end - start)
    PI_Linear_list.append(model_Linear)
    loss_list.append(np.array(Loss_record_Linear))



print('computation time ', np.array(Comp_time_list))
print('computation time mean', np.mean(np.array(Comp_time_list)))

In [ ]:
plt.plot(Loss_record_Linear)
plt.xlabel('episoid')
plt.ylabel('Loss')
plt.title('Non-Discounted Loss without penalty')

### DenseNN-PI

In [ ]:
### edge and P control
# RNN Cell to integrate state transition dynamics
class MinimalRNNCell_NNMatrix(keras.layers.Layer):

    def __init__(self, units, action_units_edge, action_units_node_p, action_units_node_i, internal_units,internal_units_ICNN,env,batchsize,**kwargs):
        self.units = units
        self.action_units_edge = action_units_edge
        self.action_units_node_p = action_units_node_p
        self.action_units_node_i = action_units_node_i
        self.state_size = units
        self.internal_units = internal_units
        self.internal_units_ICNN = internal_units_ICNN

        self.batchsize=batchsize
        self.delta_t = tf.constant(env.delta_t,dtype=tf.float32)

        self.v0 = tf.constant(env.v0,dtype=tf.float32)
        self.v1 = tf.constant(env.v1,dtype=tf.float32)
        self.state_v_ref = tf.constant(env.state_v_ref*np.ones((batchsize, action_units_node_p)),dtype=tf.float32)
        self.diag_v1 = tf.constant(env.diag_v1,dtype=tf.float32)
        self.diag_K = tf.constant(env.diag_K ,dtype=tf.float32)
        self.delta_t = tf.constant(env.delta_t ,dtype=tf.float32)
        self.dim_action = env.dim_action
        # tf.constant( ,dtype=tf.float32)
        self.dim_edge = env.dim_edge
        self.eta0 = tf.constant(env.eta0 ,dtype=tf.float32)
        self. matrix_grad_action =  tf.constant(env.matrix_grad_action,dtype=tf.float32)


        self.Penalty_action = tf.constant(env.Penalty_action ,dtype=tf.float32)
        self.diag_c = tf.constant(env.diag_c ,dtype=tf.float32)
        self.incidence = tf.constant(env.incidence ,dtype=tf.float32)
        self.w_recover =tf.constant(tf.linalg.band_part(-tf.ones((internal_units,internal_units)),0,1)\
                                        +2*tf.eye(internal_units),dtype=tf.float32)
        self.b_recover = tf.constant(tf.linalg.band_part(tf.ones((internal_units,internal_units)),0,-1)\
                                        -tf.eye(internal_units),dtype=tf.float32)

########### edge

        self.max_action_edge = tf.constant(env.max_action_edge,dtype=tf.float32)
        self.Multiply_ones_edge = tf.tile(tf.ones((action_units_edge,action_units_edge),dtype=np.float32)[None], [batchsize, 1, 1])
        self.ones_edge = tf.ones((action_units_edge,internal_units),dtype=tf.float32)

############ PI controller :P

        self.Multiply_ones_node_p = tf.tile(tf.ones((action_units_node_p,action_units_node_p),dtype=np.float32)[None], [batchsize, 1, 1])
        self.ones_node_p = tf.ones((action_units_node_p,internal_units),dtype=tf.float32)

############ PI controller :I
        self.Multiply_ones_node_i = tf.tile(tf.ones((action_units_node_i,action_units_node_i),dtype=np.float32)[None], [batchsize, 1, 1])
        self.ones_node_i = tf.ones((action_units_node_i,internal_units),dtype=tf.float32)
        self.obs_zeros = tf.zeros((1, action_units_node_p))

        self.linear_i = tf.zeros((action_units_node_i),dtype=np.float32)

        super(MinimalRNNCell_NNMatrix, self).__init__(**kwargs)

    def build(self, input_shape):
############## edge
        self.w_plus_temp0_edge =  self.add_weight(
            shape=(self.action_units_edge,self.internal_units),
            # initializer=tf.constant_initializer(0.5),
            initializer='random_normal',
            trainable=True,
            name='w_plus_temp')

        self.b_plus_temp0_edge = self.add_weight(
            shape=(self.action_units_edge,self.internal_units),
            initializer='random_normal',
            trainable=True,
            name='b_plus_temp')
        self.w_minus_temp0_edge =  self.add_weight(
            shape=(self.action_units_edge,self.internal_units),
            initializer='random_normal',
            trainable=True,
            name='w_minus_temp')

        self.b_minus_temp0_edge = self.add_weight(
            shape=(self.action_units_edge,self.internal_units),
            initializer='random_normal',
            trainable=True,
            name='b_minus_temp')


############# PI controller  : P
        self.W_p1 =  self.add_weight(
            shape=(self.action_units_node_p,self.internal_units_ICNN),
            initializer='random_normal',
            trainable=True,
            name='W_p1')
        self.b_p1 =  self.add_weight(
            shape=(self.internal_units_ICNN,),
            initializer='random_normal',
            trainable=True,
            name='b_p1')
        self.W_p2 =  self.add_weight(
            shape=(self.action_units_node_p,self.internal_units_ICNN),
            initializer='random_normal',
            trainable=True,
            name='W_p2')
        self.W_pz1 =  self.add_weight(
            shape=(self.internal_units_ICNN,self.internal_units_ICNN),
            initializer='random_normal',
            trainable=True,
            name='W_pz1')
        self.b_p2 =  self.add_weight(
            shape=(self.internal_units_ICNN,),
            initializer='random_normal',
            trainable=True,
            name='b_p2')

        self.W_p3 =  self.add_weight(
            shape=(self.action_units_node_p,1),
            initializer='random_normal',
            trainable=True,
            name='W_p3')
        self.W_pz2 =  self.add_weight(
            shape=(self.internal_units_ICNN,self.action_units_node_p),
            initializer='random_normal',
            trainable=True,
            name='W_pz2')
        self.b_p3 =  self.add_weight(
            shape=(self.action_units_node_p,),
            initializer='random_normal',
            trainable=True,
            name='b_p3')

        ######################### integral
        self.W_i1 =  self.add_weight(
            shape=(self.action_units_node_p,self.internal_units_ICNN),
            initializer='random_normal',
            trainable=True,
            name='W_i1')
        self.b_i1 =  self.add_weight(
            shape=(self.internal_units_ICNN,),
            initializer='random_normal',
            trainable=True,
            name='b_i1')
        self.W_i2 =  self.add_weight(
            shape=(self.action_units_node_p,self.internal_units_ICNN),
            initializer='random_normal',
            trainable=True,
            name='W_i2')
        self.W_iz1 =  self.add_weight(
            shape=(self.internal_units_ICNN,self.internal_units_ICNN),
            initializer='random_normal',
            trainable=True,
            name='W_iz1')
        self.b_i2 =  self.add_weight(
            shape=(self.internal_units_ICNN,),
            initializer='random_normal',
            trainable=True,
            name='b_i2')

        self.W_i3 =  self.add_weight(
            shape=(self.action_units_node_p,1),
            initializer='random_normal',
            trainable=True,
            name='W_i3')
        self.W_iz2 =  self.add_weight(
            shape=(self.internal_units_ICNN,self.action_units_node_i),
            initializer='random_normal',
            trainable=True,
            name='W_iz2')
        self.b_i3 =  self.add_weight(
            shape=(self.action_units_node_i,),
            initializer='random_normal',
            trainable=True,
            name='b_i3')


        self.built = True

    @tf.function
    def ICNN_action(self, obs):

        z1 = K.softplus(K.dot(obs, self.W_p1)+ self.b_p1)
        z2 = K.softplus(K.dot(obs, self.W_p2)+K.dot(z1, self.W_pz1)+ self.b_p2)
        z3 = K.dot(obs, self.W_p3)+K.dot(z2, self.W_pz2)+ self.b_p3
        return z3

    @tf.function
    def ICNN_actionI(self, obs):

        z1 = K.softplus(K.dot(obs, self.W_i1)+ self.b_i1)
        z2 = K.softplus(K.dot(obs, self.W_i2)+K.dot(z1, self.W_iz1)+ self.b_i2)
        z3 = K.dot(obs, self.W_i3)+K.dot(z2, self.W_iz2)+ self.b_i3
        return z3

    @tf.function
    def ICNN_edge(self, prev_state_eta):
        action_nonconstrain0_edge = K.sum(K.relu(K.dot(tf.linalg.diag(prev_state_eta-self.eta0),self.w_plus_temp0_edge)\
                                         +self.b_plus_temp0_edge)*self.w_minus_temp0_edge+self.b_minus_temp0_edge,axis=2)
        action_edge = self.max_action_edge - K.relu(self.max_action_edge-action_nonconstrain0_edge)\
                      +K.relu(-self.max_action_edge-action_nonconstrain0_edge)
        return action_edge


    def call(self, inputs, states):
        # stacked ReLU structure to represent control network
        prev_state = states[0]
        prev_state_v = prev_state[:,0:self.dim_action]
        prev_state_eta = prev_state[:, self.dim_action:self.dim_action+self.dim_edge]
        prev_state_s =  prev_state[:,self.dim_action+self.dim_edge:self.dim_action*2+self.dim_edge]

################ edge
        action_edge = self.ICNN_edge(prev_state_eta)

##################### PI: P
        obs_y = -prev_state_v+self.state_v_ref
        action_node_p = self.ICNN_action(obs_y)



##################### PI: I


        action_node_i = self.ICNN_actionI(prev_state_s)

#########################
        # calculate state on s
        dot_s = -prev_state_v + self.state_v_ref


#######################
        # integrate the state transition dynamics


        action_network = -K.dot(action_edge,tf.transpose(self.incidence))
        dot_v = K.dot((-prev_state_v+self.v0) + K.dot(action_network+action_node_p+action_node_i,self.diag_v1), self.diag_K)
        dot_eta = K.dot(prev_state_v, self.incidence)


        new_state_v = prev_state_v + self.delta_t*dot_v
        new_state_eta = prev_state_eta + self.delta_t*dot_eta
        new_state_s = prev_state_s + self.delta_t*dot_s

        next_state = tf.concat([new_state_v,  new_state_eta, new_state_s], axis=1)
        return [new_state_v-self.state_v_ref, new_state_eta, action_network, action_node_p, action_node_i], [next_state]




In [ ]:
loop_seed = 5
PI_NNMatrix_list = []
loss_list = []
loss_all_list = []

Comp_time_list = []


episodes =400 # total number of iterations to update weights
# action_units = dim_edge
# action_units_edge, action_units_node_p, action_units_node_i

units = dim_action*2 + dim_edge #dimension of each state
internal_units=20 # demension of the neural network for control policy
internal_units_NNMatrix = 20

T = 300  #Total period considered
Batch_num=300 # number of batch in each episodes
PrintUpdate=1
ref_v_upper = 6
ref_v_lower = 5

for loop in range(0,loop_seed):
    print('loop', loop)
    random.seed(datetime.now())
    start = time.time()


    cell = MinimalRNNCell_NNMatrix(units, dim_edge ,dim_action, dim_action, internal_units, internal_units_NNMatrix,env,Batch_num)
    layer = RNN(cell,return_sequences=True,stateful = True)
    input_1 = tf.keras.Input(batch_shape=(Batch_num,T,units))
    outputs = layer((input_1))
    model_NNMatrix = tf.keras.models.Model([input_1], outputs)
    model_NNMatrix.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

    x0=np.ones((Batch_num,T,units))
    y0=model_NNMatrix(x0)
    Loss_record_NNMatrix=[]
    Loss_all_record_NNMatrix=[]
    global_step = tf.Variable(0, trainable=False)
    learning_rate_initial=0.035
    decayed_lr =tf.keras.optimizers.schedules.ExponentialDecay(
        learning_rate_initial, 50, 0.7, staircase=True)
    optimizer=tf.keras.optimizers.Adam(learning_rate=decayed_lr)


    for i in range(0,episodes):
        start_in = time.time()
        cell.state_v_ref = tf.constant(np.random.uniform(ref_v_lower,ref_v_upper,(Batch_num, 1))@
                                       np.ones((1,dim_action)),dtype=tf.float32)
        initial_state1 = v0_nom*np.ones((Batch_num,dim_action)) + v0_random*np.random.uniform(0,1.,(Batch_num, dim_action))
        initial_state2 = env.eta0*np.ones((Batch_num,dim_edge))
        initial_state3 = np.zeros((Batch_num, dim_action))
        initial_state = np.hstack((initial_state1, initial_state2, initial_state3))
        layer.reset_states( initial_state)
        with tf.GradientTape(persistent=True) as tape:
            [new_state_v, new_state_eta, action_edge, action_P, action_I]=model_NNMatrix(x0)

    ##################
            loss_state1 =1* K.sum(K.relu(-new_state_eta+1))/Batch_num/units

            state_v_minus_ref = new_state_v
            loss_state3 = K.sum(K.abs(state_v_minus_ref))/Batch_num/units
            loss_action_edge = 0.01*K.sum(K.pow(action_edge, 2))/Batch_num/units
            loss_action_w = 0.05*K.sum(K.pow(action_P+action_I,2)@env.diag_c)/Batch_num/units
            loss = (loss_state3 + loss_action_edge + loss_action_w)
            loss_state_large = 10*K.sum(K.relu(K.abs(state_v_minus_ref)-5))/Batch_num/units
            loss_all = loss + loss_state_large

        grads = tape.gradient(loss, model_NNMatrix.variables)
        optimizer.apply_gradients(zip(grads, model_NNMatrix.variables))
        Loss_record_NNMatrix.append(loss)
        Loss_all_record_NNMatrix.append(loss_all)
        if i % (PrintUpdate) == 0:
            print('episode',i, 'loss_all',loss_all, 'Loss',loss, 'loss_state_large', loss_state_large)
            print('   Loss_v_minus_ref',loss_state3, 'Loss_eta',loss_state1  )
            print('   Loss_u_edge', loss_action_edge, 'loss_action_w', loss_action_w )
            print('   up/ui',K.sum(K.pow(action_P,2)@env.diag_c)/K.sum(K.pow(action_I,2)@env.diag_c))
            print('            time,',  time.time()- start_in  )



    end = time.time()
    print(end - start)
    Comp_time_list.append(end - start)
    PI_NNMatrix_list.append(model_NNMatrix)
    loss_list.append(np.array(Loss_record_NNMatrix))
    loss_all_list.append(np.array(Loss_all_record_NNMatrix))



print('computation time ', np.array(Comp_time_list))
print('computation time mean', np.mean(np.array(Comp_time_list)))


In [ ]:
plt.plot(Loss_record_NNMatrix)
plt.xlabel('episoid')
plt.ylabel('Loss')
plt.title('Non-Discounted Loss without penalty')

# Simulate

## Neural-PI

In [ ]:
# @tf.function
def Action_edge(state, model, env):

    w_plus=K.dot(tf.math.square(model.variables[0]),model.w_recover)
    b_plus=K.dot(-tf.math.square(model.variables[1]),model.b_recover)
    w_minus=K.dot(-tf.math.square(model.variables[2]),model.w_recover)
    b_minus=K.dot(-tf.math.square(model.variables[3]),model.b_recover)
    nonlinear_plus=K.sum(K.relu(K.dot(tf.linalg.diag(state-env.eta0),model.ones_edge)+b_plus)\
                    *w_plus,axis=2)
    nonlinear_minus=K.sum(K.relu(-K.dot(tf.linalg.diag(state-env.eta0),model.ones_edge)+b_minus)\
                    *w_minus,axis=2)
    action_nonconstrain0= nonlinear_plus+nonlinear_minus
    action=env.max_action_edge-np_relu(env.max_action_edge-action_nonconstrain0)\
            +np_relu(-env.max_action_edge-action_nonconstrain0)

    return action

@tf.function
def Action_ICNN_P(obs, model, env):
    with tf.GradientTape() as tape:
        tape.watch(obs)
        z1 = model.softplus_beta(K.dot(obs, model.variables[4])+ model.variables[5])
        z2 = model.softplus_beta(K.dot(obs, model.variables[6])+K.dot(z1, model.variables[7])+ model.variables[8])
        z3 = model.softplus_beta(K.dot(obs, model.variables[9])+K.dot(z2, model.variables[10])+ model.variables[11])

    return tf.squeeze(tape.batch_jacobian(z3, obs))



@tf.function
def Action_ICNN_I(obs, model, env):
    with tf.GradientTape() as tape:
        tape.watch(obs)
        z1 = model.softplus_beta(K.dot(obs, model.variables[12])+ model.variables[13])
        z2 = model.softplus_beta(K.dot(obs, model.variables[14])+K.dot(z1, model.variables[15])+ model.variables[16])
        z3 = model.softplus_beta(K.dot(obs, model.variables[17])+K.dot(z2, model.variables[18])+ model.variables[19])

    return tf.squeeze(tape.batch_jacobian(z3, obs), axis=0)


#
###########    add
def Action_ICNN(state_x, state_s, model, env):

    action_nonconstrain0 = Action_ICNN_P(-state_x+env.state_v_ref, model, env)\
                          -Action_ICNN_P(model.obs_zeros, model, env)
    action_nonconstrain1 = Action_ICNN_I(state_s,model, env)-Action_ICNN_I(model.obs_zeros, model, env)
    action_nonconstrain =  action_nonconstrain0 + action_nonconstrain1
    return action_nonconstrain, action_nonconstrain0, action_nonconstrain1


In [ ]:
# Plot the trajectory to visulize the performance of control
Trajectory_Linear = []
Trajectory_eta_Linear = []
Trajectory_s_Linear = []

init_state = np.array([[4.5481234, 4.4400673, 3.0106626, 4.648548 , 3.862493 , 3.4021516,
        4.1023088, 6.730633 , 4.2614346, 5.3563185, 5.5848107, 5.681905 ,
        3.5720792, 4.010938 , 6.539898 , 3.4076588, 6.5165396, 4.065842 ,
        6.97986  , 5.6366544 ]], dtype=np.float32)
# (v0_nom*np.ones(dim_action,dtype=np.float32) +
              # v0_random*np.random.uniform(0,1.,(dim_action)).astype(np.float32)).reshape((1, -1))

# v0.reshape((1, -1))
linear_coff_s = np.ones((1,dim_action),dtype=np.float32)*(2)
linear_coff_x = linear_coff= np.ones((1,dim_action),dtype=np.float32)*10

x = init_state
action_s = np.zeros((1,dim_action),dtype=np.float32)

env.set_state(x)

Trajectory_Linear.append(x)
Trajectory_eta_Linear.append(env.state_eta)

SimulationLength=760
Record_u_Linear=[]
Record_Loss_Linear=[]
Record_action_network_Linear=[]
Loss_Linear=0
state_s = np.zeros((1,dim_action),dtype=np.float32)
Record_up=[]
Record_ui=[]
record_grad_ui = []
Trajectory_s_Linear.append(env.state_s)

for i in range(SimulationLength):


    action_edgefeedback = Action_edge(env.state_eta, PI_SCNN_list[0], env)

    u, up, ui = Action_ICNN(x, state_s, PI_SCNN_list[0], env)
    next_state_s, next_state_v, next_state_eta, r, action_network= env.step_edge_WoCost(action_edgefeedback, up, ui)

    Loss_Linear+=r
    x = next_state_v
    state_s = next_state_s
    Trajectory_Linear.append(x)
    Trajectory_eta_Linear.append(next_state_eta)
    Trajectory_s_Linear.append(next_state_s)
    Record_u_Linear.append(np.squeeze(u))
    Record_up.append(np.squeeze(up))
    Record_ui.append(np.squeeze(ui))
    Record_action_network_Linear.append(np.squeeze(action_network))
    Record_Loss_Linear.append(np.squeeze(r))
    record_grad_ui.append(np.squeeze(env.calc_grad_action(ui)[1]))

Trajectory_Linear = np.squeeze(np.asarray(Trajectory_Linear))
Trajectory_s_Linear = np.squeeze(np.asarray(Trajectory_s_Linear))
record_grad_ui = np.squeeze(np.asarray(record_grad_ui))
Record_u_Linear = np.squeeze(np.asarray(Record_u_Linear))
fig = plt.figure(figsize=(11,10), dpi=100)

plt.subplot(4,2,1)


TimeRecord=np.arange(1,SimulationLength+1)
TimeRecord=env.delta_t*TimeRecord
plt.plot(TimeRecord,Record_u_Linear)
plt.xlabel('time(s)')
plt.ylabel('Action_total')

plt.subplot(4,2,2)
TimeRecord=np.arange(1,SimulationLength+1)
TimeRecord=env.delta_t*TimeRecord
plt.plot(TimeRecord,Record_action_network_Linear)
plt.xlabel('time(s)')
plt.ylabel('Action_network')

plt.subplot(4,2,3)

TimeRecord=np.arange(1,SimulationLength+2)
TimeRecord=env.delta_t*TimeRecord
plt.plot(TimeRecord,Trajectory_Linear)

plt.xlabel('time(s)')
plt.ylabel('speed')

Trajectory_eta_Linear=np.squeeze(np.asarray(Trajectory_eta_Linear))

plt.subplot(4,2,4)
plt.plot(TimeRecord,Trajectory_eta_Linear)
plt.xlabel('time(s)')
plt.ylabel('position')

plt.subplot(4,2,5)
TimeRecord=np.arange(1,SimulationLength+1)
TimeRecord=env.delta_t*TimeRecord
plt.plot(TimeRecord,Record_up)
plt.xlabel('time(s)')
plt.ylabel('Action u_p')


plt.subplot(4,2,6)
TimeRecord=np.arange(1,SimulationLength+1)
TimeRecord=env.delta_t*TimeRecord
plt.plot(TimeRecord,Record_ui)
plt.xlabel('time(s)')
plt.ylabel('Action u_i')
fig.tight_layout()

plt.subplot(4,2,7)
TimeRecord=np.arange(1,SimulationLength+2)
TimeRecord=env.delta_t*TimeRecord
plt.plot(TimeRecord,Trajectory_s_Linear)

plt.xlabel('time(s)')
plt.ylabel('state_s')



plt.subplot(4,2,8)
TimeRecord=np.arange(1,SimulationLength+1)
TimeRecord=env.delta_t*TimeRecord
plt.plot(TimeRecord,record_grad_ui)
plt.xlabel('time(s)')
plt.ylabel('grad u_i')
fig.tight_layout()




## Linear-PI

In [ ]:
# @tf.function
def Action_Linear_edge(state, model, env):
    action_nonconstrain0 = (state-env.eta0)*abs(model.variables[0])
    action=env.max_action_edge-np_relu(env.max_action_edge-action_nonconstrain0)\
            +np_relu(-env.max_action_edge-action_nonconstrain0)
    return action

@tf.function
def Action_Linear_P(obs, model, env):
    z1 = K.dot(obs,  model.variables[1])
    return z1



@tf.function
def Action_Linear_I(obs, model, env):
    z1 = K.dot(obs,  model.variables[2])
    return z1

###########    add
def Action_Linear(state_x, state_s, model, env):

    action_nonconstrain0 = Action_Linear_P(-state_x+env.state_v_ref, model, env)
    action_nonconstrain1 = Action_Linear_I(state_s,model, env)
    action_nonconstrain =  action_nonconstrain0 + action_nonconstrain1

    return action_nonconstrain, action_nonconstrain0, action_nonconstrain1


In [ ]:
# Plot the trajectory to visulize the performance of control

Trajectory_Linear = []
Trajectory_eta_Linear = []
Trajectory_s_Linear = []

init_state = np.array([[4.5481234, 4.4400673, 3.0106626, 4.648548 , 3.862493 , 3.4021516,
        4.1023088, 6.730633 , 4.2614346, 5.3563185, 5.5848107, 5.681905 ,
        3.5720792, 4.010938 , 6.539898 , 3.4076588, 6.5165396, 4.065842 ,
        6.97986  , 5.6366544 ]], dtype=np.float32)
# (v0_nom*np.ones(dim_action,dtype=np.float32) +
              # v0_random*np.random.uniform(0,1.,(dim_action)).astype(np.float32)).reshape((1, -1))

# v0.reshape((1, -1))
linear_coff_s = np.ones((1,dim_action),dtype=np.float32)*(2)
linear_coff_x = linear_coff= np.ones((1,dim_action),dtype=np.float32)*10

x = init_state
action_s = np.zeros((1,dim_action),dtype=np.float32)

env.set_state(x)

Trajectory_Linear.append(x)
Trajectory_eta_Linear.append(env.state_eta)

SimulationLength=760
Record_u_Linear=[]
Record_Loss_Linear=[]
Record_action_network_Linear=[]
Loss_Linear=0
state_s = np.zeros((1,dim_action),dtype=np.float32)
Record_up=[]
Record_ui=[]
record_grad_ui = []
Trajectory_s_Linear.append(env.state_s)

for i in range(SimulationLength):

    id_model =3

    action_edgefeedback = Action_Linear_edge(env.state_eta,  PI_Linear_list[id_model], env)

    u, up, ui = Action_Linear(x, state_s,  PI_Linear_list[id_model], env)

    next_state_s, next_state_v, next_state_eta, r, action_network= env.step_edge_WoCost(action_edgefeedback, up, ui)

    Loss_Linear+=r
    x = next_state_v
    state_s = next_state_s
    Trajectory_Linear.append(x)
    Trajectory_eta_Linear.append(next_state_eta)
    Trajectory_s_Linear.append(next_state_s)
    Record_u_Linear.append(np.squeeze(u))
    Record_up.append(np.squeeze(up))
    Record_ui.append(np.squeeze(ui))
    Record_action_network_Linear.append(np.squeeze(action_network))
    Record_Loss_Linear.append(np.squeeze(r))
    record_grad_ui.append(np.squeeze(env.calc_grad_action(ui)[1]))

Trajectory_Linear = np.squeeze(np.asarray(Trajectory_Linear))
Trajectory_s_Linear = np.squeeze(np.asarray(Trajectory_s_Linear))
record_grad_ui = np.squeeze(np.asarray(record_grad_ui))
Record_u_Linear = np.squeeze(np.asarray(Record_u_Linear))
fig = plt.figure(figsize=(11,10), dpi=100)

plt.subplot(4,2,1)


TimeRecord=np.arange(1,SimulationLength+1)
TimeRecord=env.delta_t*TimeRecord
plt.plot(TimeRecord,Record_u_Linear)
plt.xlabel('time(s)')
plt.ylabel('Action_total')

plt.subplot(4,2,2)
TimeRecord=np.arange(1,SimulationLength+1)
TimeRecord=env.delta_t*TimeRecord
plt.plot(TimeRecord,Record_action_network_Linear)
plt.xlabel('time(s)')
plt.ylabel('Action_network')

plt.subplot(4,2,3)

TimeRecord=np.arange(1,SimulationLength+2)
TimeRecord=env.delta_t*TimeRecord
plt.plot(TimeRecord,Trajectory_Linear)

plt.xlabel('time(s)')
plt.ylabel('speed')

Trajectory_eta_Linear=np.squeeze(np.asarray(Trajectory_eta_Linear))

plt.subplot(4,2,4)
plt.plot(TimeRecord,Trajectory_eta_Linear)
plt.xlabel('time(s)')
plt.ylabel('position')

plt.subplot(4,2,5)
TimeRecord=np.arange(1,SimulationLength+1)
TimeRecord=env.delta_t*TimeRecord
plt.plot(TimeRecord,Record_up)
plt.xlabel('time(s)')
plt.ylabel('Action u_p')


plt.subplot(4,2,6)
TimeRecord=np.arange(1,SimulationLength+1)
TimeRecord=env.delta_t*TimeRecord
plt.plot(TimeRecord,Record_ui)
plt.xlabel('time(s)')
plt.ylabel('Action u_i')
fig.tight_layout()

plt.subplot(4,2,7)
TimeRecord=np.arange(1,SimulationLength+2)
TimeRecord=env.delta_t*TimeRecord
plt.plot(TimeRecord,Trajectory_s_Linear)

plt.xlabel('time(s)')
plt.ylabel('state_s')



plt.subplot(4,2,8)
TimeRecord=np.arange(1,SimulationLength+1)
TimeRecord=env.delta_t*TimeRecord
plt.plot(TimeRecord,record_grad_ui)
plt.xlabel('time(s)')
plt.ylabel('grad u_i')
fig.tight_layout()


## DenseNN-PI

In [ ]:
# @tf.function
def Action_edge(state, model, env):
    action_nonconstrain0= K.sum(K.relu(K.dot(tf.linalg.diag(state-env.eta0),model.variables[0])+model.variables[1])*\
                  model.variables[2]+model.variables[3],axis=2)
    action=env.max_action_edge-np_relu(env.max_action_edge-action_nonconstrain0)\
            +np_relu(-env.max_action_edge-action_nonconstrain0)
    return action

@tf.function
def Action_NNMatrix_P(obs, model, env):
    z1 = K.softplus(K.dot(obs, model.variables[4])+ model.variables[5])
    z2 = K.softplus(K.dot(obs, model.variables[6])+K.dot(z1, model.variables[7])+ model.variables[8])
    z3 = K.dot(obs, model.variables[9])+K.dot(z2, model.variables[10])+ model.variables[11]
    return z3



@tf.function
def Action_NNMatrix_I(obs, model, env):

    z1 = K.softplus(K.dot(obs, model.variables[12])+ model.variables[13])
    z2 = K.softplus(K.dot(obs, model.variables[14])+K.dot(z1, model.variables[15])+ model.variables[16])
    z3 = K.dot(obs, model.variables[17])+K.dot(z2, model.variables[18])+ model.variables[19]
    return z3

#
###########    add
def Action_NNMatrix(state_x, state_s, model, env):

    action_nonconstrain0 = Action_NNMatrix_P(-state_x+env.state_v_ref, model, env)
    action_nonconstrain1 = Action_NNMatrix_I(state_s,model, env)
    action_nonconstrain =  action_nonconstrain0 + action_nonconstrain1
    return action_nonconstrain, action_nonconstrain0, action_nonconstrain1


In [ ]:
# Plot the trajectory to visulize the performance of control

Trajectory_Linear = []
Trajectory_eta_Linear = []
Trajectory_s_Linear = []

init_state = np.array([[4.5481234, 4.4400673, 3.0106626, 4.648548 , 3.862493 , 3.4021516,
        4.1023088, 6.730633 , 4.2614346, 5.3563185, 5.5848107, 5.681905 ,
        3.5720792, 4.010938 , 6.539898 , 3.4076588, 6.5165396, 4.065842 ,
        6.97986  , 5.6366544 ]], dtype=np.float32)
# (v0_nom*np.ones(dim_action,dtype=np.float32) +
              # v0_random*np.random.uniform(0,1.,(dim_action)).astype(np.float32)).reshape((1, -1))

# v0.reshape((1, -1))
linear_coff_s = np.ones((1,dim_action),dtype=np.float32)*(2)
linear_coff_x = linear_coff= np.ones((1,dim_action),dtype=np.float32)*10

x = init_state
action_s = np.zeros((1,dim_action),dtype=np.float32)

env.set_state(x)

Trajectory_Linear.append(x)
Trajectory_eta_Linear.append(env.state_eta)

SimulationLength=760
Record_u_Linear=[]
Record_Loss_Linear=[]
Record_action_network_Linear=[]
Loss_Linear=0
state_s = np.zeros((1,dim_action),dtype=np.float32)
Record_up=[]
Record_ui=[]
record_grad_ui = []
Trajectory_s_Linear.append(env.state_s)

for i in range(SimulationLength):


    id_model = 0
    action_edgefeedback = Action_edge(env.state_eta, PI_NNMatrix_list[id_model], env)
    u, up, ui = Action_NNMatrix(x, state_s,PI_NNMatrix_list[id_model], env)
    next_state_s, next_state_v, next_state_eta, r, action_network= env.step_edge_WoCost(action_edgefeedback, up, ui)

    Loss_Linear+=r
    x = next_state_v
    state_s = next_state_s
    Trajectory_Linear.append(x)
    Trajectory_eta_Linear.append(next_state_eta)
    Trajectory_s_Linear.append(next_state_s)
    Record_u_Linear.append(np.squeeze(u))
    Record_up.append(np.squeeze(up))
    Record_ui.append(np.squeeze(ui))
    Record_action_network_Linear.append(np.squeeze(action_network))
    Record_Loss_Linear.append(np.squeeze(r))
    record_grad_ui.append(np.squeeze(env.calc_grad_action(ui)[1]))

Trajectory_Linear = np.squeeze(np.asarray(Trajectory_Linear))
Trajectory_s_Linear = np.squeeze(np.asarray(Trajectory_s_Linear))
record_grad_ui = np.squeeze(np.asarray(record_grad_ui))
Record_u_Linear = np.squeeze(np.asarray(Record_u_Linear))
fig = plt.figure(figsize=(11,10), dpi=100)

plt.subplot(4,2,1)


TimeRecord=np.arange(1,SimulationLength+1)
TimeRecord=env.delta_t*TimeRecord
plt.plot(TimeRecord,Record_u_Linear)
plt.xlabel('time(s)')
plt.ylabel('Action_total')

plt.subplot(4,2,2)
TimeRecord=np.arange(1,SimulationLength+1)
TimeRecord=env.delta_t*TimeRecord
plt.plot(TimeRecord,Record_action_network_Linear)
plt.xlabel('time(s)')
plt.ylabel('Action_network')

plt.subplot(4,2,3)

TimeRecord=np.arange(1,SimulationLength+2)
TimeRecord=env.delta_t*TimeRecord
plt.plot(TimeRecord,Trajectory_Linear)

plt.xlabel('time(s)')
plt.ylabel('speed')

Trajectory_eta_Linear=np.squeeze(np.asarray(Trajectory_eta_Linear))

plt.subplot(4,2,4)
plt.plot(TimeRecord,Trajectory_eta_Linear)
plt.xlabel('time(s)')
plt.ylabel('position')

plt.subplot(4,2,5)
TimeRecord=np.arange(1,SimulationLength+1)
TimeRecord=env.delta_t*TimeRecord
plt.plot(TimeRecord,Record_up)
plt.xlabel('time(s)')
plt.ylabel('Action u_p')


plt.subplot(4,2,6)
TimeRecord=np.arange(1,SimulationLength+1)
TimeRecord=env.delta_t*TimeRecord
plt.plot(TimeRecord,Record_ui)
plt.xlabel('time(s)')
plt.ylabel('Action u_i')
fig.tight_layout()

plt.subplot(4,2,7)
TimeRecord=np.arange(1,SimulationLength+2)
TimeRecord=env.delta_t*TimeRecord
plt.plot(TimeRecord,Trajectory_s_Linear)

plt.xlabel('time(s)')
plt.ylabel('state_s')



plt.subplot(4,2,8)
TimeRecord=np.arange(1,SimulationLength+1)
TimeRecord=env.delta_t*TimeRecord
plt.plot(TimeRecord,record_grad_ui)
plt.xlabel('time(s)')
plt.ylabel('grad u_i')
fig.tight_layout()
